In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv('data/dataset.csv')
data = data.sort_values(['timestamp'])

In [3]:
train = data[:80000]
test = data[80000:]

In [4]:
train.head()

,user_id,item_id,rating,timestamp
217,259,255,4,874724710
83968,259,286,4,874724727
43030,259,298,4,874724754
21399,259,185,4,874724781
82658,259,173,4,874724843


In [5]:
test.head()

,user_id,item_id,rating,timestamp
1346,3,245,1,889237247
27978,3,355,3,889237247
1260,3,335,1,889237269
38673,3,322,3,889237269
3761,3,323,2,889237269


In [6]:
# Задача: Обучить модель так, чтобы мера была больше 0.1

### Обработка данных
Сначала необходимо составить матрицу **user-item**, где индексами являются идентификаторы пользователей, а колонками - идентификаторы продукта. Нам нужна также валидационная выборка, чтобы на ней мы подобрали оптимальные гиперпараметры (размер валидационной выборки будет составлять 20% от тренировочной).

In [7]:
validation = train[:16000]
train1 = train[16000:]

In [8]:
def matrix(train):
    
    users_number = len(data['user_id'].unique()) # количество уникальных пользователей
    items_number = len(data['item_id'].unique()) # количество уникальных товаров

    matrix = pd.DataFrame(np.zeros((users_number, items_number)), 
                            index = sorted(data['user_id'].unique()), 
                            columns = sorted(data['item_id'].unique()))


    for i in train.index:
        user = train['user_id'][i]
        item = train['item_id'][i]
        rating = train['rating'][i]
        matrix[item][user] = 1
    return matrix
    
users_matrix = matrix(train1)
users_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


В итоге получилась таблица, где на пересечении id пользователя и продукта стоит 1, если пользователь оценил данный продукт, и 0 иначе (выставляются оценки только те, которые есть в обучающей выборке).

Матрица на первый вгзгляд выглядит разреженной, чтобы это проверить, посмотрим на количество пропусков.

In [9]:
spaces = 0
for i in users_matrix:
    spaces += len(users_matrix[users_matrix[i] == 0.0])
print('Процент пропущенных данных равен {:%}'.format(spaces/(len(data['user_id'].unique())*len(data['item_id'].unique()))))


Процент пропущенных данных равен 95.969286%


Присутствует около 5% данных, поэтому будет целесообразно использовать структуру данных **sparse table**

In [10]:
# создание разреженной таблицы
from scipy.sparse import csr_matrix

users_matrix_sparse = csr_matrix(users_matrix.values)
users_matrix_sparse

<944x1682 sparse matrix of type '<class 'numpy.float64'>'
	with 64000 stored elements in Compressed Sparse Row format>

## Описание использованной модели и гиперпараметров

Существует множество моделей рекомендательных систем, однако наиболее приемлемыми для разреженных таблиц являются модели, основанные на матричной факторизации. SVD разложение не годится, так как оно плохо работает с разреженными таблицами. Разложение, которое используется в модели **LightFM**, выделяет латентные факторы пользователей и продуктов, что позволяет снизить размерность данных. Данная модель работает также с неявным рейтингом. Ниже представлено разложение матрицы:


***Обозначения***:

Мн. пользователей | Мн. продуктов | Пользователь | Продукт | Фактор пользователя | Фактор продукта | Характеристика пользователей | Характеристика продуктов
--- | --- | --- |  --- | --- | --- | --- | ---
$U$ | $I$ | $u$ | $i$ | $f_u$ | $f_i$ | $e_f^U$ | $e_f^I$


Латентные признаки пользователя | Латентные признаки продукта | Член смещения пользователя | Член смещения продукта
--- | --- | --- | ---
$q_u = \sum_{j \in f_u}e_j^U$ | $p_i = \sum_{j \in f_i}e_j^I$ | $b_u = \sum_{j \in f_u}b_j^U$ | $b_u = \sum_{j \in f_u}b_j^I$


Тогда предсказание для пользователя $u$ и продукта $i$ считается как *скалярное произведение латентных факторов* пользователя и продукта с добавлением члена смещения по продукту и пользователю (это нужно для L2 регуляризации, так как пользователи могут ставить оценки предвзято):

$$\hat{r_{ui}} = f(q_u\cdot p_i + b_u + b_i)$$

В качестве loss функции, которую мы будем минимизировать, я выбрала **WRAP**, так как она обычно дает наилучший результат. Минимизация будет происходит с помощью стохастического градиентного спуска.

Количество факторов, которые будет выделять модель, как и другие **гиперпараметры**, найдем с помощью подбора. Для этого нужна функция расчета метрики `metric_function`, которую будем максимизировать. Функция `optimize_params` перебирает гиперпараметры из заданного диапазона и возвращает те, на которых модель выдала наилучший результат.

### Расчет метрики

In [11]:
def average_precision(actual, recommended, k=30):
    ap_sum = 0
    hits = 0
    for i in range(k):
        product_id = recommended[i] if i < len(recommended) else None
        if product_id is not None and product_id in actual:
            hits += 1
            ap_sum += hits / (i + 1)
    return ap_sum / k


def normalized_average_precision(actual, recommended, k=30):
    actual = set(actual)
    if len(actual) == 0:
        return 0.0

    ap = average_precision(actual, recommended, k=k)
    ap_ideal = average_precision(actual, list(actual)[:k], k=k)
    return ap / ap_ideal

### Функция создания рекомендаций

In [12]:
def recommend(user, model, data = users_matrix, number_of_recommendations = 30):
    
    # вычисляем скалярное произведение получившихся факторов
    predictions = pd.Series(model.predict(int(user), np.arange(data.shape[1])), index = data.columns)
    data_transpose = np.transpose(data)
    
    # отдельно выносим те продукты, которые есть в тренировочной выборке
    contained_items = list(data_transpose.loc[data_transpose.loc[:, user] != 0].index)
    
    # сортируем скалярные произведения, тогда на 1-м месте для пользователя u будет находиться продукт, 
    # который подходит ему больше всего
    recommended = list(predictions.sort_values(ascending = False).index)
    
    # определяем индексы предсказанных продуктов, убирая из них те, что есть в тренировочной выборке
    recommended = [r for r in recommended if r not in contained_items][:number_of_recommendations]
    
    return recommended

### Вычисление метрики

In [13]:
def metric_function(model, test):
    scores = []
    for user in test['user_id'].unique():
        actual = list(test[test['user_id'] == user]['item_id'])
        recommended = recommend(user, model)
    
        scores.append(normalized_average_precision(actual, recommended))

    return np.mean(scores)

### Функция для подбора гиперпараметров

In [14]:
from lightfm import LightFM

def optimize_params(metric_function, params, train = users_matrix_sparse, test = validation):
    
    # диапазон гиперпараметров
    learning_rate_range, no_components_range, alpha_range = params

    optimal_metric = 0
    optimal_params = {'learning_rate' : 0,
                      'no_components' : 0,
                      'alpha' : 0}
    
    for learning_rate in learning_rate_range:
        for no_components in no_components_range:
            for alpha in alpha_range:
                    
                # создаем модель
                model = LightFM(loss='warp',
                                no_components=no_components,
                                random_state=1234,
                                learning_rate=learning_rate,
                                user_alpha=alpha)
                # тренируем модель
                model.fit(train,
                          epochs=10,
                          num_threads=16)
                
                # считаем метрику на текущих гиперпараметрах
                new_metric = metric_function(model, test)
                    
                # если текущая метрика лучше, чем текущая наибольшая метрика, то обновляем словарь гиперпараметров
                if new_metric > optimal_metric:
                    optimal_metric = new_metric
                    optimal_params['learning_rate'] = learning_rate
                    optimal_params['no_components'] = no_components
                    optimal_params['alpha'] = alpha
    return optimal_params


/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


### Находим наиболее подходящие гиперпараметры

In [15]:
# диапазон гиперпараметров
params = [np.arange(10**-4, 10**-1, 0.1),  # learning_rate
          np.arange(5, 100),               # no_components
          np.arange(10**-6, 10**-2, 0.01)] # alpha

optimal_params = optimize_params(metric_function, params)
optimal_params

{'learning_rate': 0.0001, 'no_components': 63, 'alpha': 1e-06}

### Итоговое тестирование

Теперь можно обучить модель с найденными гиперпараметрами на первоначальной тренировочной выборке.

In [16]:
# итоговая тренировочная выборка 
users_matrix_final = matrix(train)
users_matrix_final_sparse = csr_matrix(users_matrix_final.values)
users_matrix_final_sparse

<944x1682 sparse matrix of type '<class 'numpy.float64'>'
	with 80000 stored elements in Compressed Sparse Row format>

In [23]:
# создаем модель с найденными гиперпараметрами
model = LightFM(loss='warp',
                no_components=optimal_params['no_components'],
                random_state=1234,
                learning_rate=optimal_params['learning_rate'],
                user_alpha=optimal_params['alpha'])

# обучаем модель
model.fit(users_matrix_final_sparse,
                  epochs=10,
                  num_threads=16)

## Метрика на тестовых данных:

In [24]:
scores = []
for user in tqdm(test['user_id'].unique()):
    actual = list(test[test['user_id'] == user]['item_id'])
    recommended = recommend(user, model)
    
    scores.append(normalized_average_precision(actual, recommended))

np.mean(scores)

100%|██████████| 301/301 [00:01<00:00, 293.70it/s]


0.1650523762615745